In [1]:
!nvidia-smi

Mon May 25 10:51:04 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 432.00       Driver Version: 432.00       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN X (Pascal)   WDDM  | 00000000:01:00.0  On |                  N/A |
| 23%   34C    P8    18W / 250W |    912MiB / 12288MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm
import tensorflow as tf
import os
from os.path import join
import random
import lightgbm as lgb
import time
from tensorflow.python.framework import ops
from tensorflow.python.ops import math_ops
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Conv2DTranspose, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Activation, concatenate, Input, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras import Model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
# from tensorflow.nn import seperable_conv2d
import warnings
 
warnings.filterwarnings("ignore")

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
# model_v4 / external feature / atrous conv / spatial pyramid pooling
def atrous_conv(img_layer, start_neurons):
    conv1 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(img_layer)
    conv1 = BatchNormalization()(conv1)
    conv2 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same", dilation_rate=2)(img_layer)
    conv2 = BatchNormalization()(conv2)
    conv3 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same", dilation_rate=3)(img_layer)
    conv3 = BatchNormalization()(conv3)
    print('atrous conv3 shape:', conv3.shape)
    pool = AveragePooling2D((2,2))(img_layer)
    conv4 = Conv2DTranspose(start_neurons, (3, 3), strides=(2, 2), padding="same")(pool)

    total = concatenate([conv1, conv2, conv3, conv4])
    print('atrous concat:', total.shape)
    total = Conv2D(start_neurons, (1,1), activation='relu', padding='valid')(total)
    print('final atrous shape:', total.shape)
    total = Dropout(0.2)(total)

    return total

def conv_block_3(img_layer, start_neurons):
    conv1 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(img_layer)
    conv1 = BatchNormalization()(conv1)
    conv2 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(conv1)
    conv2 = BatchNormalization()(conv2)
    conv3 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(conv2)
    conv3 = BatchNormalization()(conv3)

    return conv3

def conv_residual(img_layer, start_neurons):
    conv1 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(img_layer)
    conv1 = BatchNormalization()(conv1)
    conv2 = conv_block_3(conv1, start_neurons)
    conv2 = BatchNormalization()(conv2)
    conv3 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(conv1 + conv2)
    conv3 = BatchNormalization()(conv3)

    return conv3

def model_v4(input_layer, start_neurons, ext_neurons):
    # divide raw feature to image feature and external feature
    img_layer = input_layer[:,:,:,:9]
    ext_layer = input_layer[:,:,:,9:]
    print('input_layer shape:', input_layer.shape)
    print('img_layer shape:', img_layer.shape)
    print('ext_layer shape:', ext_layer.shape)

    # 40 x 40 -> 20 x 20
    print('img_layer shape:', img_layer.shape)
    conv1 = conv_residual(img_layer, start_neurons * 1)
    print('conv1 shape:', conv1.shape)
    pool1 = MaxPooling2D((2, 2))(conv1)
    print('maxpool 1 shape:', pool1.shape)
    pool1 = Dropout(0.2)(pool1)

    # 20 x 20 -> 10 x 10
    conv2 = conv_residual(pool1, start_neurons * 2)
    print('conv2 shape:', conv2.shape)
    pool2 = MaxPooling2D((2, 2))(conv2)
    print('maxpool 2 shape:', pool2.shape)
    pool2 = Dropout(0.2)(pool2)

    # 10 x 10 
    convm = atrous_conv(pool2, start_neurons * 4)
    print('convm shape:', convm.shape)

    # 10 x 10 -> 20 x 20
    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv2 = concatenate([deconv2, conv2])
    print('concat 2:', uconv2.shape)
    uconv2 = conv_residual(uconv2, start_neurons * 2)
    print('upconv2 shape:', uconv2.shape)
    uconv2 = Dropout(0.2)(uconv2)

    # 20 x 20 -> 40 x 40
    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = conv_residual(uconv1, start_neurons * 1)
    print('upconv1 shape:', uconv1.shape)
    uconv1 = Dropout(0.2)(uconv1)
    
    # process the external feature
    ext_output = Conv2D(ext_neurons, (1,1), padding="valid", activation='relu')(ext_layer)
    ext_output = BatchNormalization()(ext_output)
    print('ext 1 shape:', ext_output.shape)
    ext_output = Conv2D(ext_neurons, (1,1), padding="valid", activation='relu')(ext_layer)
    ext_output = BatchNormalization()(ext_output)
    print('ext 2 shape:', ext_output.shape)
    ext_output = Dropout(0.2)(ext_output)

    output_layer = concatenate([uconv1, ext_output])
    print('final concat shape:', output_layer.shape)
    output_layer = Conv2D(1, (1,1), padding="valid", activation='relu')(output_layer)
    print('output shape:', output_layer.shape)
    
    return output_layer

input_layer = Input((40, 40, 19))
output_layer = model_v4(input_layer, 36, 40)
model = Model(input_layer, output_layer)

input_layer shape: (None, 40, 40, 19)
img_layer shape: (None, 40, 40, 9)
ext_layer shape: (None, 40, 40, 10)
img_layer shape: (None, 40, 40, 9)
conv1 shape: (None, 40, 40, 36)
maxpool 1 shape: (None, 20, 20, 36)
conv2 shape: (None, 20, 20, 72)
maxpool 2 shape: (None, 10, 10, 72)
atrous conv3 shape: (None, 10, 10, 144)
atrous concat: (None, 10, 10, 576)
final atrous shape: (None, 10, 10, 144)
convm shape: (None, 10, 10, 144)
concat 2: (None, 20, 20, 144)
upconv2 shape: (None, 20, 20, 72)
upconv1 shape: (None, 40, 40, 36)
ext 1 shape: (None, 40, 40, 40)
ext 2 shape: (None, 40, 40, 40)
final concat shape: (None, 40, 40, 76)
output shape: (None, 40, 40, 1)


In [4]:
# training_set.ftr 파일에 맞는 input 생성 코드
def input_generator_ftr_train():
    # train feather 파일 로드 및 orbit, subset 조합 생성
    TRAIN_FILE = './fe_data/train_mid.ftr'
    train = pd.read_feather(TRAIN_FILE)
    file_cnt = 0
    temp_info = train[['orbit', 'subset']].drop_duplicates().sort_values(['orbit', 'subset'])
    orbit = temp_info['orbit'].tolist()
    subset = temp_info['subset'].tolist()
    del temp_info
    size = len(orbit)
  
  # 하나의 이미지 데이터에 해당하는 것만 3차원 변환 및 피처/라벨 생성
    for i in range(size):
        one_img = train.loc[(train['orbit'] == orbit[i]) & (train['subset'] == subset[i])].sort_values('pixel')
        one_img['type1'] = one_img.loc[one_img['type_1'] == 0, 'type_1'] + 1
        one_img['type2'] = one_img.loc[one_img['type_1'] == 1, 'type_1']
        one_img['type3'] = one_img.loc[one_img['type_1'] == 2, 'type_1'] - 1
        one_img['type4'] = one_img.loc[one_img['type_1'] == 3, 'type_1'] - 2
        one_img['type_2'] /= 100
        one_img.fillna(0, inplace=True)
        one_img['label'] = one_img['precipitation']
        one_img = np.array(one_img.drop(['type_1', 'orbit', 'subset', 'pixel', 'precipitation'], axis=1)).reshape([40,40,20])
        target = one_img[:,:,-1].reshape(40,40,1)
        cutoff_labels = np.where(target < 0,0, target)
        feature = one_img[:,:,:-1]
        # if (cutoff_labels > 0).sum() < 50: continue
        yield(feature, cutoff_labels)
        file_cnt += 1

train_dataset = tf.data.Dataset.from_generator(input_generator_ftr_train, (tf.float32, tf.float32),
                                               (tf.TensorShape([40,40,19]),tf.TensorShape([40,40,1])))
train_dataset = train_dataset.batch(256).prefetch(1)

In [5]:
# custom loss fuction (maeOverFscore)
def recall_m(y_true, y_pred):
    true_positives = tf.dtypes.cast(K.sum(K.round(K.clip(y_true * y_pred, 0, 1))), dtype=tf.float32)
    possible_positives = tf.dtypes.cast(K.sum(K.round(K.clip(y_true, 0, 1))), dtype=tf.float32)
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = tf.dtypes.cast(K.sum(K.round(K.clip(y_true * y_pred, 0, 1))), dtype=tf.float32)
    predicted_positives = tf.dtypes.cast(K.sum(K.round(K.clip(y_pred, 0, 1))), dtype=tf.float32)
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    idx_one_true = tf.greater_equal(y_true, 0.1)
    y_true = tf.where(idx_one_true == True, 1, 0)
    idx_one_pred = tf.greater_equal(y_pred, 0.1)
    y_pred = tf.where(idx_one_pred == True, 1, 0)
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def mae(y_true, y_pred):
    over_threshold = tf.greater_equal(y_true, 0.1)
    return K.mean(tf.concat([(math_ops.abs(y_pred[over_threshold] - y_true[over_threshold]) * 5),
                             math_ops.abs(y_pred[~over_threshold] - y_true[~over_threshold])], axis=0), axis=-1)

def maeOverFscore(y_true, y_pred):
    y_true = tf.reshape(y_true, [-1])
    y_pred = tf.reshape(y_pred, [-1])
    return mae(y_true, y_pred) / f1_m(y_true, y_pred)

In [6]:
# 모델 옵션 설정
WEIGHT_DIR = './checkpoint/ver_loss_new/'
LOG_DIR = './log/'
logging = TensorBoard(log_dir=LOG_DIR)
checkpoint = ModelCheckpoint(WEIGHT_DIR + 'v02_ep{epoch:02d}-loss{loss:.5f}.ckpt', mode='min',
                             monitor='loss', save_weights_only=True, period=1, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=1, verbose=1)
early_stopping = EarlyStopping(monitor='loss', min_delta=0, patience=2, verbose=1)
callbacks = [logging, checkpoint, reduce_lr, early_stopping]
adam = tf.keras.optimizers.Adam(learning_rate=1.0e-04)
model.compile(loss=maeOverFscore, optimizer=adam, metrics=[mae])

In [7]:
# 이전에 훈련한 weight를 이어서 훈련할 경우 실행
latest = tf.train.latest_checkpoint(WEIGHT_DIR)
print(latest)
model.load_weights(latest)

./checkpoint/ver_loss_new/v01_ep03-loss1.74201.ckpt


In [8]:
start = time.time()
epochs = 12
model_history = model.fit(train_dataset, epochs = epochs, verbose=1,
                          callbacks=callbacks)
print('{}epoch 파일 학습 시간:'.format(epochs), round((time.time()-start)/60, 2))

Epoch 1/12


    178/Unknown - 168s 168s/step - loss: 1.5380 - mae: 0.410 - 286s 143s/step - loss: 1.3914 - mae: 0.354 - 415s 138s/step - loss: 1.4160 - mae: 0.322 - 536s 134s/step - loss: 1.3105 - mae: 0.309 - 656s 131s/step - loss: 1.2724 - mae: 0.315 - 776s 129s/step - loss: 1.2347 - mae: 0.313 - 896s 128s/step - loss: 1.2521 - mae: 0.323 - 1016s 127s/step - loss: 1.2552 - mae: 0.32 - 1136s 126s/step - loss: 1.2565 - mae: 0.32 - 1255s 126s/step - loss: 1.2266 - mae: 0.32 - 1376s 125s/step - loss: 1.2219 - mae: 0.32 - 1498s 125s/step - loss: 1.2136 - mae: 0.32 - 1625s 125s/step - loss: 1.1968 - mae: 0.32 - 1746s 125s/step - loss: 1.1859 - mae: 0.32 - 1868s 125s/step - loss: 1.1768 - mae: 0.31 - 1995s 125s/step - loss: 1.1877 - mae: 0.30 - 2117s 125s/step - loss: 1.1785 - mae: 0.30 - 2240s 124s/step - loss: 1.1655 - mae: 0.30 - 2362s 124s/step - loss: 1.1556 - mae: 0.30 - 2484s 124s/step - loss: 1.1406 - mae: 0.30 - 2606s 124s/step - loss: 1.1301 - mae: 0.30 - 2729s 124s/step - loss: 1.1214 - mae:

194/299 [==================>...........] - ETA: 11:54:00 - loss: 1.1214 - mae: 0.381 - ETA: 10:51:25 - loss: 1.0129 - mae: 0.328 - ETA: 10:29:05 - loss: 1.0155 - mae: 0.297 - ETA: 10:16:38 - loss: 0.9326 - mae: 0.284 - ETA: 10:08:25 - loss: 0.9162 - mae: 0.291 - ETA: 10:02:56 - loss: 0.8915 - mae: 0.288 - ETA: 9:57:33 - loss: 0.9107 - mae: 0.298 - ETA: 9:52:56 - loss: 0.9191 - mae: 0.30 - ETA: 9:49:12 - loss: 0.9285 - mae: 0.30 - ETA: 9:45:48 - loss: 0.9076 - mae: 0.29 - ETA: 9:42:17 - loss: 0.9048 - mae: 0.30 - ETA: 9:39:07 - loss: 0.8979 - mae: 0.30 - ETA: 9:36:33 - loss: 0.8851 - mae: 0.30 - ETA: 9:34:06 - loss: 0.8731 - mae: 0.29 - ETA: 9:31:27 - loss: 0.8639 - mae: 0.29 - ETA: 9:29:06 - loss: 0.8706 - mae: 0.28 - ETA: 9:26:47 - loss: 0.8661 - mae: 0.28 - ETA: 9:24:25 - loss: 0.8576 - mae: 0.28 - ETA: 9:22:09 - loss: 0.8489 - mae: 0.28 - ETA: 9:19:58 - loss: 0.8388 - mae: 0.27 - ETA: 9:17:40 - loss: 0.8310 - mae: 0.27 - ETA: 9:15:26 - loss: 0.8258 - mae: 0.28 - ETA: 9:13:08 - loss:

298/299 [============================>.] - ETA: 3:27:02 - loss: 1.1163 - mae: 0.54 - ETA: 3:25:02 - loss: 1.1157 - mae: 0.54 - ETA: 3:23:03 - loss: 1.1139 - mae: 0.54 - ETA: 3:21:03 - loss: 1.1125 - mae: 0.54 - ETA: 3:19:03 - loss: 1.1119 - mae: 0.54 - ETA: 3:17:05 - loss: 1.1108 - mae: 0.54 - ETA: 3:15:07 - loss: 1.1096 - mae: 0.54 - ETA: 3:13:09 - loss: 1.1089 - mae: 0.54 - ETA: 3:11:13 - loss: 1.1088 - mae: 0.54 - ETA: 3:09:15 - loss: 1.1070 - mae: 0.54 - ETA: 3:07:16 - loss: 1.1059 - mae: 0.54 - ETA: 3:05:18 - loss: 1.1050 - mae: 0.54 - ETA: 3:03:18 - loss: 1.1044 - mae: 0.53 - ETA: 3:01:20 - loss: 1.1042 - mae: 0.53 - ETA: 2:59:25 - loss: 1.1031 - mae: 0.53 - ETA: 2:57:28 - loss: 1.1017 - mae: 0.53 - ETA: 2:55:29 - loss: 1.1004 - mae: 0.53 - ETA: 2:53:30 - loss: 1.0990 - mae: 0.53 - ETA: 2:51:33 - loss: 1.0969 - mae: 0.53 - ETA: 2:49:34 - loss: 1.0951 - mae: 0.53 - ETA: 2:47:35 - loss: 1.0935 - mae: 0.53 - ETA: 2:45:36 - loss: 1.0935 - mae: 0.53 - ETA: 2:43:37 - loss: 1.0928 - mae

194/299 [==================>...........] - ETA: 14:35:45 - loss: 0.9183 - mae: 0.364 - ETA: 12:08:07 - loss: 0.8257 - mae: 0.313 - ETA: 11:18:20 - loss: 0.8202 - mae: 0.283 - ETA: 10:51:36 - loss: 0.7558 - mae: 0.270 - ETA: 10:36:13 - loss: 0.7499 - mae: 0.277 - ETA: 10:25:32 - loss: 0.7335 - mae: 0.275 - ETA: 10:16:46 - loss: 0.7529 - mae: 0.285 - ETA: 10:09:48 - loss: 0.7593 - mae: 0.290 - ETA: 10:04:01 - loss: 0.7652 - mae: 0.287 - ETA: 9:58:58 - loss: 0.7486 - mae: 0.283 - ETA: 9:54:32 - loss: 0.7467 - mae: 0.29 - ETA: 9:55:50 - loss: 0.7426 - mae: 0.28 - ETA: 9:52:00 - loss: 0.7328 - mae: 0.28 - ETA: 9:47:58 - loss: 0.7210 - mae: 0.28 - ETA: 9:44:26 - loss: 0.7137 - mae: 0.27 - ETA: 9:40:55 - loss: 0.7197 - mae: 0.27 - ETA: 9:37:30 - loss: 0.7157 - mae: 0.27 - ETA: 9:35:17 - loss: 0.7091 - mae: 0.26 - ETA: 9:33:03 - loss: 0.7014 - mae: 0.26 - ETA: 9:30:57 - loss: 0.6930 - mae: 0.26 - ETA: 9:28:18 - loss: 0.6867 - mae: 0.26 - ETA: 9:25:40 - loss: 0.6832 - mae: 0.26 - ETA: 9:22:58 -

...

194/299 [==================>...........] - ETA: 11:54:30 - loss: 0.6049 - mae: 0.328 - ETA: 10:57:05 - loss: 0.5332 - mae: 0.283 - ETA: 10:36:22 - loss: 0.5011 - mae: 0.253 - ETA: 10:25:26 - loss: 0.4647 - mae: 0.241 - ETA: 10:17:26 - loss: 0.4650 - mae: 0.246 - ETA: 10:11:51 - loss: 0.4596 - mae: 0.245 - ETA: 10:06:58 - loss: 0.4753 - mae: 0.255 - ETA: 10:02:52 - loss: 0.4821 - mae: 0.260 - ETA: 9:59:19 - loss: 0.4814 - mae: 0.256 - ETA: 9:56:15 - loss: 0.4732 - mae: 0.25 - ETA: 9:53:14 - loss: 0.4754 - mae: 0.25 - ETA: 9:50:34 - loss: 0.4751 - mae: 0.25 - ETA: 9:47:44 - loss: 0.4703 - mae: 0.25 - ETA: 9:45:09 - loss: 0.4598 - mae: 0.25 - ETA: 9:42:41 - loss: 0.4521 - mae: 0.24 - ETA: 9:40:19 - loss: 0.4517 - mae: 0.24 - ETA: 9:37:49 - loss: 0.4485 - mae: 0.24 - ETA: 9:35:25 - loss: 0.4452 - mae: 0.23 - ETA: 9:33:05 - loss: 0.4402 - mae: 0.23 - ETA: 9:30:49 - loss: 0.4351 - mae: 0.23 - ETA: 9:28:27 - loss: 0.4327 - mae: 0.23 - ETA: 9:26:10 - loss: 0.4331 - mae: 0.23 - ETA: 9:23:56 - l

298/299 [============================>.] - ETA: 3:32:00 - loss: 0.7454 - mae: 0.49 - ETA: 3:29:58 - loss: 0.7449 - mae: 0.49 - ETA: 3:27:55 - loss: 0.7435 - mae: 0.48 - ETA: 3:25:53 - loss: 0.7427 - mae: 0.48 - ETA: 3:23:50 - loss: 0.7419 - mae: 0.48 - ETA: 3:21:48 - loss: 0.7411 - mae: 0.48 - ETA: 3:19:46 - loss: 0.7404 - mae: 0.48 - ETA: 3:17:45 - loss: 0.7400 - mae: 0.48 - ETA: 3:15:45 - loss: 0.7400 - mae: 0.48 - ETA: 3:13:45 - loss: 0.7387 - mae: 0.48 - ETA: 3:11:43 - loss: 0.7379 - mae: 0.48 - ETA: 3:09:41 - loss: 0.7372 - mae: 0.48 - ETA: 3:07:40 - loss: 0.7367 - mae: 0.48 - ETA: 3:05:37 - loss: 0.7367 - mae: 0.48 - ETA: 3:03:35 - loss: 0.7359 - mae: 0.48 - ETA: 3:01:32 - loss: 0.7347 - mae: 0.48 - ETA: 2:59:29 - loss: 0.7338 - mae: 0.48 - ETA: 2:57:26 - loss: 0.7328 - mae: 0.48 - ETA: 2:55:24 - loss: 0.7313 - mae: 0.47 - ETA: 2:53:21 - loss: 0.7299 - mae: 0.47 - ETA: 2:51:18 - loss: 0.7285 - mae: 0.47 - ETA: 2:49:16 - loss: 0.7285 - mae: 0.47 - ETA: 2:47:13 - loss: 0.7277 - mae

194/299 [==================>...........] - ETA: 12:16:27 - loss: 0.5850 - mae: 0.323 - ETA: 11:08:11 - loss: 0.5160 - mae: 0.280 - ETA: 10:44:28 - loss: 0.4838 - mae: 0.251 - ETA: 10:31:28 - loss: 0.4490 - mae: 0.238 - ETA: 10:22:47 - loss: 0.4500 - mae: 0.244 - ETA: 10:16:16 - loss: 0.4452 - mae: 0.243 - ETA: 10:10:59 - loss: 0.4604 - mae: 0.252 - ETA: 10:06:57 - loss: 0.4678 - mae: 0.257 - ETA: 10:03:10 - loss: 0.4665 - mae: 0.253 - ETA: 9:59:54 - loss: 0.4595 - mae: 0.250 - ETA: 9:56:48 - loss: 0.4621 - mae: 0.25 - ETA: 9:53:50 - loss: 0.4626 - mae: 0.25 - ETA: 9:50:54 - loss: 0.4585 - mae: 0.25 - ETA: 9:48:12 - loss: 0.4480 - mae: 0.25 - ETA: 9:45:41 - loss: 0.4403 - mae: 0.24 - ETA: 9:43:06 - loss: 0.4392 - mae: 0.24 - ETA: 9:40:36 - loss: 0.4360 - mae: 0.23 - ETA: 9:38:11 - loss: 0.4329 - mae: 0.23 - ETA: 9:35:47 - loss: 0.4281 - mae: 0.23 - ETA: 9:33:26 - loss: 0.4231 - mae: 0.23 - ETA: 9:31:03 - loss: 0.4210 - mae: 0.23 - ETA: 9:28:47 - loss: 0.4222 - mae: 0.23 - ETA: 9:26:28 -

194/299 [==================>...........] - ETA: 12:15:10 - loss: 0.5705 - mae: 0.320 - ETA: 11:01:26 - loss: 0.5020 - mae: 0.276 - ETA: 10:36:38 - loss: 0.4683 - mae: 0.248 - ETA: 10:21:12 - loss: 0.4353 - mae: 0.235 - ETA: 10:11:36 - loss: 0.4359 - mae: 0.241 - ETA: 10:05:29 - loss: 0.4319 - mae: 0.240 - ETA: 10:00:51 - loss: 0.4466 - mae: 0.249 - ETA: 9:56:29 - loss: 0.4532 - mae: 0.254 - ETA: 9:52:09 - loss: 0.4513 - mae: 0.25 - ETA: 9:48:19 - loss: 0.4437 - mae: 0.24 - ETA: 9:44:57 - loss: 0.4468 - mae: 0.25 - ETA: 9:41:42 - loss: 0.4475 - mae: 0.25 - ETA: 9:38:46 - loss: 0.4437 - mae: 0.25 - ETA: 9:36:09 - loss: 0.4334 - mae: 0.24 - ETA: 9:33:30 - loss: 0.4260 - mae: 0.24 - ETA: 9:30:52 - loss: 0.4243 - mae: 0.23 - ETA: 9:28:18 - loss: 0.4213 - mae: 0.23 - ETA: 9:25:55 - loss: 0.4184 - mae: 0.23 - ETA: 9:23:28 - loss: 0.4139 - mae: 0.23 - ETA: 9:21:07 - loss: 0.4089 - mae: 0.23 - ETA: 9:18:43 - loss: 0.4074 - mae: 0.23 - ETA: 9:16:16 - loss: 0.4089 - mae: 0.23 - ETA: 9:14:03 - los

298/299 [============================>.] - ETA: 3:29:49 - loss: 0.7181 - mae: 0.48 - ETA: 3:27:49 - loss: 0.7177 - mae: 0.48 - ETA: 3:25:49 - loss: 0.7163 - mae: 0.48 - ETA: 3:23:49 - loss: 0.7155 - mae: 0.48 - ETA: 3:21:47 - loss: 0.7148 - mae: 0.48 - ETA: 3:19:46 - loss: 0.7139 - mae: 0.47 - ETA: 3:17:46 - loss: 0.7133 - mae: 0.47 - ETA: 3:15:45 - loss: 0.7129 - mae: 0.47 - ETA: 3:13:43 - loss: 0.7129 - mae: 0.47 - ETA: 3:11:42 - loss: 0.7117 - mae: 0.47 - ETA: 3:09:40 - loss: 0.7109 - mae: 0.47 - ETA: 3:07:39 - loss: 0.7101 - mae: 0.47 - ETA: 3:05:38 - loss: 0.7097 - mae: 0.47 - ETA: 3:03:36 - loss: 0.7096 - mae: 0.47 - ETA: 3:01:34 - loss: 0.7088 - mae: 0.47 - ETA: 2:59:33 - loss: 0.7077 - mae: 0.47 - ETA: 2:57:34 - loss: 0.7067 - mae: 0.47 - ETA: 2:55:40 - loss: 0.7057 - mae: 0.47 - ETA: 2:53:39 - loss: 0.7042 - mae: 0.47 - ETA: 2:51:39 - loss: 0.7028 - mae: 0.47 - ETA: 2:49:39 - loss: 0.7015 - mae: 0.46 - ETA: 2:47:39 - loss: 0.7016 - mae: 0.46 - ETA: 2:45:39 - loss: 0.7008 - mae

## train, test set에 결과 붙이기

In [12]:
# cv,ftr 파일에 맞는 input 생성 코드
def input_generator_ftr_test():
    # cv feather 파일 로드 및 orbit, subset 조합 생성
    TEST_FILE = './ftr_data/train_mid.ftr'
    test = pd.read_feather(TEST_FILE)
    file_cnt = 0
    temp_info = test[['orbit', 'subset']].drop_duplicates()
    orbit = temp_info['orbit'].tolist()
    subset = temp_info['subset'].tolist()
    del temp_info
    size = len(orbit)
  
  # 하나의 이미지 데이터에 해당하는 것만 3차원 변환 및 피처/라벨 생성
    for i in range(size):
        one_img = test.loc[(test['orbit'] == orbit[i]) & (test['subset'] == subset[i])].sort_values('pixel')
        one_img['type1'] = one_img.loc[one_img['type_1'] == 0, 'type_1'] + 1
        one_img['type2'] = one_img.loc[one_img['type_1'] == 1, 'type_1']
        one_img['type3'] = one_img.loc[one_img['type_1'] == 2, 'type_1'] - 1
        one_img['type4'] = one_img.loc[one_img['type_1'] == 3, 'type_1'] - 2
        one_img['type_2'] /= 100
        one_img.fillna(0, inplace=True)
        
#         one_img['label'] = one_img['precipitation']
        one_img = np.array(one_img.drop(['type_1', 'orbit', 'subset', 'pixel', 'precipitation'], axis=1)).reshape([40,40,19])
#         one_img = np.array(one_img.drop(['type_1', 'orbit', 'subset', 'pixel'], axis=1)).reshape([40,40,19])
#         target = one_img[:,:,-1].reshape(40,40,1)
#         cutoff_labels = np.where(target < 0,0, target)
        feature = one_img[:,:,:-1]
        # if (cutoff_labels > 0).sum() < 50: continue
#         yield(feature, cutoff_labels)
        yield(feature)
        file_cnt += 1

test_dataset = tf.data.Dataset.from_generator(input_generator_ftr_test, tf.float32,
                                               tf.TensorShape([40,40,19]))
test_dataset = test_dataset.batch(256).prefetch(1)

In [13]:
WEIGHT_DIR = './checkpoint/ver_loss_new/'
latest = tf.train.latest_checkpoint(WEIGHT_DIR)
print(latest)
model.load_weights(latest)
adam = tf.keras.optimizers.Adam(learning_rate=1.0e-05)
model.compile(loss=maeOverFscore, optimizer=adam, metrics=[mae])
result = model.predict(test_dataset)
result.shape

./checkpoint/ver_loss_new/v01_ep12-loss0.71871.ckpt


InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument:  UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc1 in position 18: invalid start byte
Traceback (most recent call last):

  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\ops\script_ops.py", line 221, in __call__
    ret = func(*args)

  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\data\ops\dataset_ops.py", line 585, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "<ipython-input-12-da2df882af5d>", line 5, in input_generator_ftr_test
    test = pd.read_feather(TEST_FILE)

  File "C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\feather_format.py", line 103, in read_feather
    return feather.read_feather(path, columns=columns, use_threads=bool(use_threads))

  File "C:\ProgramData\Anaconda3\lib\site-packages\pyarrow\feather.py", line 214, in read_feather
    reader = FeatherReader(source)

  File "C:\ProgramData\An

# Cross Validation 평가

In [0]:
# cv,ftr 파일에 맞는 input 생성 코드
def input_generator_ftr_cv():
  # cv feather 파일 로드 및 orbit, subset 조합 생성
  CV_FILE = './ftr_data/cv.ftr'
  cv = pd.read_feather(CV_FILE)
  file_cnt = 0
  temp_info = cv[['orbit', 'subset']].drop_duplicates()
  orbit = temp_info['orbit'].tolist()
  subset = temp_info['subset'].tolist()
  del temp_info
  size = len(orbit)
  
  # 하나의 이미지 데이터에 해당하는 것만 3차원 변환 및 피처/라벨 생성
  for i in range(size):
    one_img = cv.loc[(cv['orbit'] == orbit[i]) & (cv['subset'] == subset[i])].sort_values('pixel')
    one_img = np.array(one_img.drop(['orbit', 'subset', 'pixel'], axis=1)).reshape([40,40,15])
    target = one_img[:,:,-1].reshape(40,40,1)
    cutoff_labels = np.where(target < 0,0, target)
    feature = one_img[:,:,:-1]
    # if (cutoff_labels > 0).sum() < 50: continue
    yield(feature, cutoff_labels)
    file_cnt += 1

cv_dataset = tf.data.Dataset.from_generator(input_generator_ftr_cv, (tf.float32, tf.float32),
                                            (tf.TensorShape([40,40,14]),tf.TensorShape([40,40,1])))
cv_dataset = cv_dataset.batch(64)

In [0]:
# model_version1
def conv_block_3(img_layer, start_neurons):
  conv1 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(img_layer)
  conv1 = BatchNormalization()(conv1)
  conv2 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(conv1)
  conv2 = BatchNormalization()(conv2)
  conv3 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(conv2)
  conv3 = BatchNormalization()(conv3)

  return conv3

def conv_residual(img_layer, start_neurons):
  conv1 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(img_layer)
  conv1 = BatchNormalization()(conv1)
  conv2 = conv_block_3(conv1, start_neurons)
  conv2 = BatchNormalization()(conv2)
  conv3 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(conv1 + conv2)
  conv3 = BatchNormalization()(conv3)

  return conv3

def model_v1(input_layer, start_neurons):
    # divide raw feature to image feature and external feature
    img_layer = input_layer[:,:,:,:9]
    ext_layer = input_layer[:,:,:,9:]
    print('input_layer shape:', input_layer.shape)
    print('img_layer shape:', img_layer.shape)
    print('ext_layer shape:', ext_layer.shape)

    # 40 x 40 -> 20 x 20
    print('img_layer shape:', img_layer.shape)
    conv1 = conv_residual(img_layer, start_neurons * 1)
    print('conv1 shape:', conv1.shape)
    pool1 = MaxPooling2D((2, 2))(conv1)
    print('maxpool 1 shape:', pool1.shape)
    pool1 = Dropout(0.25)(pool1)

    # 20 x 20 -> 10 x 10
    conv2 = conv_residual(pool1, start_neurons * 2)
    print('conv2 shape:', conv2.shape)
    pool2 = MaxPooling2D((2, 2))(conv2)
    print('maxpool 2 shape:', pool2.shape)
    pool2 = Dropout(0.25)(pool2)

    # 10 x 10 
    convm = conv_residual(pool2, start_neurons * 4)
    print('convm shape:', convm.shape)

    # 10 x 10 -> 20 x 20
    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv2 = concatenate([deconv2, conv2])
    uconv2 = conv_residual(uconv2, start_neurons * 2)
    print('upconv2 shape:', uconv2.shape)
    uconv2 = Dropout(0.25)(uconv2)

    # 20 x 20 -> 40 x 40
    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = conv_residual(uconv1, start_neurons * 1)
    print('upconv1 shape:', uconv1.shape)
    uconv1 = Dropout(0.25)(uconv1)
    output_layer = Conv2D(1, (1,1), padding="same", activation='relu')(uconv1)
    print('output shape:', output_layer.shape)
    
    return output_layer

input_layer = Input((40, 40, 17))
output_layer = model_v1(input_layer, 32)
model = Model(input_layer, output_layer)

In [0]:
# model version 2
def conv_block_3(img_layer, start_neurons):
    conv1 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(img_layer)
    conv1 = BatchNormalization()(conv1)
    conv2 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(conv1)
    conv2 = BatchNormalization()(conv2)
    conv3 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(conv2)
    conv3 = BatchNormalization()(conv3)

    return conv3

def conv_residual(img_layer, start_neurons):
    conv1 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(img_layer)
    conv1 = BatchNormalization()(conv1)
    conv2 = conv_block_3(conv1, start_neurons)
    conv2 = BatchNormalization()(conv2)
    conv3 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(conv1 + conv2)
    conv3 = BatchNormalization()(conv3)

    return conv3

def model_v2(input_layer, start_neurons, ext_neurons):
    # divide raw feature to image feature and external feature
    img_layer = input_layer[:,:,:,:9]
    ext_layer = input_layer[:,:,:,9:]
    print('input_layer shape:', input_layer.shape)
    print('img_layer shape:', img_layer.shape)
    print('ext_layer shape:', ext_layer.shape)

    # 40 x 40 -> 20 x 20
    print('img_layer shape:', img_layer.shape)
    conv1 = conv_residual(img_layer, start_neurons * 1)
    print('conv1 shape:', conv1.shape)
    pool1 = MaxPooling2D((2, 2))(conv1)
    print('maxpool 1 shape:', pool1.shape)
    pool1 = Dropout(0.25)(pool1)

    # 20 x 20 -> 10 x 10
    conv2 = conv_residual(pool1, start_neurons * 2)
    print('conv2 shape:', conv2.shape)
    pool2 = MaxPooling2D((2, 2))(conv2)
    print('maxpool 2 shape:', pool2.shape)
    pool2 = Dropout(0.25)(pool2)

    # 10 x 10 
    convm = conv_residual(pool2, start_neurons * 4)
    print('convm shape:', convm.shape)

    # 10 x 10 -> 20 x 20
    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv2 = concatenate([deconv2, conv2])
    print('concat 2:', uconv2.shape)
    uconv2 = conv_residual(uconv2, start_neurons * 2)
    print('upconv2 shape:', uconv2.shape)
    uconv2 = Dropout(0.25)(uconv2)

    # 20 x 20 -> 40 x 40
    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = conv_residual(uconv1, start_neurons * 1)
    print('upconv1 shape:', uconv1.shape)
    uconv1 = Dropout(0.25)(uconv1)
    
    # process the external feature
    ext_output = Conv2D(ext_neurons, (1,1), padding="valid", activation='relu')(ext_layer)
    ext_output = BatchNormalization()(ext_output)
    print('ext 1 shape:', ext_output.shape)
    # ext_output = Dropout(0.25)(ext_output)
    ext_output = Conv2D(ext_neurons, (1,1), padding="valid", activation='relu')(ext_layer)
    ext_output = BatchNormalization()(ext_output)
    print('ext 2 shape:', ext_output.shape)

    output_layer = concatenate([uconv1, ext_output])
    print('final concat shape:', output_layer.shape)

    output_layer = Conv2D(16, (1,1), padding="valid", activation='relu')(output_layer)
    print('middle output shape:', output_layer.shape)
    output_layer = Conv2D(1, (1,1), padding="valid", activation='relu')(output_layer)
    print('output shape:', output_layer.shape)
    
    return output_layer

input_layer = Input((40, 40, 17))
output_layer = model_v2(input_layer, 32, 64)
model = Model(input_layer, output_layer)

In [0]:
# model_v3 / external feature / atrous conv / 

def atrous_conv(img_layer, start_neurons):
  conv1 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(img_layer)
  conv1 = BatchNormalization()(conv1)
  conv2 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same", dilation_rate=2)(img_layer)
  conv2 = BatchNormalization()(conv2)
  conv3 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same", dilation_rate=3)(img_layer)
  conv3 = BatchNormalization()(conv3)
  print('atrous conv3 shape:', conv3.shape)
  pool = AveragePooling2D((2,2))(img_layer)
  print('pool shape:', pool.shape)
  conv4 = Conv2DTranspose(start_neurons, (3, 3), strides=(2, 2), padding="same")(pool)
  print('interpolate shape:', conv4.shape)

  total = concatenate([conv1, conv2, conv3, conv4])
  print('atrous concat:', total.shape)
  total = Conv2D(start_neurons, (1,1), activation='relu', padding='valid')(total)
  print('final atrous shape:', total.shape)
  total = Dropout(0.5)(total)

  return total

def conv_block_2(img_layer, start_neurons):
  conv1 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(img_layer)
  conv1 = BatchNormalization()(conv1)
  conv2 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(conv1)
  conv2 = BatchNormalization()(conv2)

  return conv2

def conv_residual(img_layer, start_neurons):
  conv1 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(img_layer)
  conv1 = BatchNormalization()(conv1)
  conv2 = conv_block_2(conv1, start_neurons)
  conv2 = BatchNormalization()(conv2)
  conv3 = Conv2D(start_neurons, (3, 3), activation="relu", padding="same")(conv1 + conv2)
  conv3 = BatchNormalization()(conv3)

  return conv3

def model_v3(input_layer, start_neurons, ext_neurons):
    # divide raw feature to image feature and external feature
    img_layer = input_layer[:,:,:,:9]
    ext_layer = input_layer[:,:,:,9:]
    print('input_layer shape:', input_layer.shape)
    print('img_layer shape:', img_layer.shape)
    print('ext_layer shape:', ext_layer.shape)

    # 40 x 40 -> 20 x 20
    print('img_layer shape:', img_layer.shape)
    conv1 = conv_residual(img_layer, start_neurons * 1)
    print('conv1 shape:', conv1.shape)
    pool1 = MaxPooling2D((2, 2))(conv1)
    print('maxpool 1 shape:', pool1.shape)
    pool1 = Dropout(0.5)(pool1)

    # 20 x 20 -> 10 x 10
    conv2 = conv_residual(pool1, start_neurons * 2)
    print('conv2 shape:', conv2.shape)
    pool2 = MaxPooling2D((2, 2))(conv2)
    print('maxpool 2 shape:', pool2.shape)
    pool2 = Dropout(0.5)(pool2)

    # 10 x 10 
    convm = atrous_conv(pool2, start_neurons * 4)
    print('convm shape:', convm.shape)

    # 10 x 10 -> 20 x 20
    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv2 = concatenate([deconv2, conv2])
    print('concat 2:', uconv2.shape)
    uconv2 = conv_residual(uconv2, start_neurons * 2)
    print('upconv2 shape:', uconv2.shape)
    uconv2 = Dropout(0.5)(uconv2)

    # 20 x 20 -> 40 x 40
    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = conv_residual(uconv1, start_neurons * 1)
    print('upconv1 shape:', uconv1.shape)
    uconv1 = Dropout(0.5)(uconv1)
    
    # process the external feature
    ext_output = Conv2D(ext_neurons, (1,1), padding="valid", activation='relu')(ext_layer)
    ext_output = BatchNormalization()(ext_output)
    print('ext 1 shape:', ext_output.shape)
    ext_output = Conv2D(ext_neurons, (1,1), padding="valid", activation='relu')(ext_layer)
    ext_output = BatchNormalization()(ext_output)
    print('ext 2 shape:', ext_output.shape)
    ext_output = Dropout(0.5)(ext_output)

    output_layer = concatenate([uconv1, ext_output])
    print('final concat shape:', output_layer.shape)

    output_layer = Conv2D(1, (1,1), padding="valid", activation='relu')(output_layer)
    print('output shape:', output_layer.shape)
    
    return output_layer

input_layer = Input((40, 40, 14))
output_layer = model_v3(input_layer, 32, 32)
model = Model(input_layer, output_layer)

input_layer shape: (None, 40, 40, 14)
img_layer shape: (None, 40, 40, 9)
ext_layer shape: (None, 40, 40, 5)
img_layer shape: (None, 40, 40, 9)
conv1 shape: (None, 40, 40, 32)
maxpool 1 shape: (None, 20, 20, 32)
conv2 shape: (None, 20, 20, 64)
maxpool 2 shape: (None, 10, 10, 64)
atrous conv3 shape: (None, 10, 10, 128)
pool shape: (None, 5, 5, 64)
interpolate shape: (None, 10, 10, 128)
atrous concat: (None, 10, 10, 512)
final atrous shape: (None, 10, 10, 128)
convm shape: (None, 10, 10, 128)
concat 2: (None, 20, 20, 128)
upconv2 shape: (None, 20, 20, 64)
upconv1 shape: (None, 40, 40, 32)
ext 1 shape: (None, 40, 40, 32)
ext 2 shape: (None, 40, 40, 32)
final concat shape: (None, 40, 40, 64)
output shape: (None, 40, 40, 1)


In [0]:
# 원본 npy 파일에 맞는 input 생성 코드
def input_generator_npy():
  train_basic_path = 'raw_data'
  train_path_list = sorted(glob.glob(train_basic_path + '/train*'))
  print('디렉토리 개수:', len(train_path_list))
  file_num = 0
  for path in train_path_list:
    files = sorted(glob.glob(path + '/*'))
    print('총 파일 개수:', file_num)
    for file in files:
      data = np.load(file)
      target = data[:, :, -1].reshape(40,40,1)
      cutoff_labels = np.where(target < 0,0, target)
      feature = data[:,:,:-1]
      if (cutoff_labels > 0).sum() < 50: continue
      yield(feature, cutoff_labels)
      file_num += 1

train_dataset = tf.data.Dataset.from_generator(input_generator_npy, (tf.float32, tf.float32),
                                               (tf.TensorShape([40,40,14]),tf.TensorShape([40,40,1])))
train_dataset = train_dataset.batch(8).prefetch(1)